In [1]:
import numpy as np
import time
import cv2
import os

In [2]:
rng = np.random.default_rng(1)

In [3]:
folder = "./data/"

img_files = [os.path.join(folder, f) for f in os.listdir(folder) if os.path.isfile(os.path.join(folder, f))]

bases = []
actives = []

for _file in img_files:
    img = cv2.imread(_file, cv2.IMREAD_GRAYSCALE).astype(np.uint8)
    rnd = (rng.random(img.shape, dtype = np.float32) * 255).astype(np.uint8)
    bases.append(img)
    actives.append(rnd)

In [5]:
np.random.seed(0)

def mat_runner(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        b = bases[i]
        a = actives[i]
        start_time = time.perf_counter()
        f(b, a)
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def mat_runner_float(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        b = bases[i].astype(np.float32)
        a = actives[i].astype(np.float32)
        opacity = rng.random(1, dtype = np.float32)
        start_time = time.perf_counter()
        f(b, a, opacity)
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def vec_runner_int(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        b = bases[i].flatten()
        a = actives[i].flatten()
        opacity = rng.random(1, dtype = np.float32).astype(np.uint8)
        start_time = time.perf_counter()
        f(b, a, opacity)
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def vec_runner_float(bases, actives, f):
    total_time = 0
    for i in range(len(bases)):
        b = bases[i].flatten().astype(np.float32)
        a = actives[i].flatten().astype(np.float32)
        opacity = rng.random(1, dtype = np.float32)
        start_time = time.perf_counter()
        f(b, a, opacity)
        end_time = time.perf_counter()
        del a
        del b
        total_time += (end_time - start_time) * 1000
    return total_time

def timer(input1, input2, f, runner):
    runs = 10
    times = []
    for _ in range(runs):
        times.append(runner(input1, input2, f))
    times = np.array(times)
    print(f"{f.__name__}")
    print(f"{np.average(times)}ms +/- {np.std(times)}ms")

In [5]:
### NUMPY

In [ ]:
def dissolve_blend_8_np(base, active, opacity):
  return np.where((opacity) - (((np.random.randint(1, 2147483647, base.shape) % (100)) + (1)) // (100)) >= 0, active, base)

In [6]:
def darken_blend_8_np(base, active):
  return np.where(np.greater(base, active), active, base)

In [7]:
def color_burn_8_np(base, active):
  return np.where(np.equal(active, 0), 255, (255) - (((255) - (base)) // (active)))

In [8]:
def lighten_blend_8_np(base, active):
  return np.where(np.less(base, active), active, base)

In [9]:
def color_dodge_8_np(base, active):
  return np.where(np.equal(active, 255), 255, (base) // ((255) - (active)))

In [10]:
def overlay_blend_8_np(base, active):
  return np.where(np.greater_equal(base, 128), ((((2) * (base)) + (base)) - ((((2) * (base)) * (base)) // (255))) - (255), (((2) * (base)) * (base)) // (255))

In [11]:
def multiply_blend_8_np(base, active):
  return ((base) * (active)) // (255)

In [12]:
def linear_burn_8_np(base, active):
  return ((base) + (active)) - (255)

In [13]:
def screen_blend_8_np(base, active):
  return ((base) + (active)) - (((base) * (active)) // (255))

In [14]:
def linear_dodge_8_np(base, active):
  return (base) + (active)

In [16]:
def normal_blend_f_np(base, active, opacity):
  return ((opacity) * (active)) + (((1) - (opacity)) * (base))

In [17]:
def normal_blend_8_np(base, active, opacity):
  return ((opacity) * (active)) + (((255) - (opacity)) * (base))

In [18]:
timer(bases, actives, darken_blend_8_np, mat_runner)

darken_blend_8_np
56.14498183131218ms +/- 0.7112932226440771ms


In [19]:
timer(bases, actives, color_burn_8_np, mat_runner)

/tmp/ipykernel_2928722/3293817414.py:2: RuntimeWarning: divide by zero encountered in floor_divide
  return np.where(np.equal(active, 0), 255, 255 - (255 - base) // active)


color_burn_8_np
43.834238359704614ms +/- 0.6384348110282989ms


In [20]:
timer(bases, actives, lighten_blend_8_np, mat_runner)

lighten_blend_8_np
55.87240308523178ms +/- 0.5146395649558829ms


In [21]:
timer(bases, actives, color_dodge_8_np, mat_runner)

color_dodge_8_np
41.16864916868508ms +/- 0.543032939644061ms


In [22]:
timer(bases, actives, overlay_blend_8_np, mat_runner)

overlay_blend_8_np
27.25212574005127ms +/- 0.579266697866349ms


In [23]:
timer(bases, actives, multiply_blend_8_np, mat_runner)

multiply_blend_8_np
3.1837795861065388ms +/- 0.20199960962185773ms


In [24]:
timer(bases, actives, linear_burn_8_np, mat_runner)

linear_burn_8_np
2.4688614532351494ms +/- 0.1701552338449329ms


In [25]:
timer(bases, actives, screen_blend_8_np, mat_runner)

screen_blend_8_np
4.887284012511373ms +/- 0.2459709687968512ms


In [26]:
timer(bases, actives, linear_dodge_8_np, mat_runner)

linear_dodge_8_np
1.9644997082650661ms +/- 0.1557456839519575ms


In [27]:
timer(bases, actives, normal_blend_f_np, vec_runner_float)

normal_blend_f_np
15.81684397533536ms +/- 0.5334233436003933ms


In [28]:
timer(bases, actives, normal_blend_8_np, vec_runner_int)

normal_blend_8_np
2.662988007068634ms +/- 0.18669919775283733ms


In [10]:
timer(bases, actives, dissolve_blend_8_np, mat_runner_float)

dissolve_blend_8_np
105.60254282318056ms +/- 4.329192122020533ms
